# Solver Benchmark - Finding All Solutions

## Z3

The `Z3` solver returns only one valid solution.
We try two ways to find/count all satisfying interpretations (models):
- Enumeration-based: Enumerate all possible assignments and check if they satisfy all assertions. This is possible in `Z3` with a substitution and simplification mechanism, but looks a bit awkward.
- [Solver-based](https://stackoverflow.com/questions/13395391/z3-finding-all-satisfying-models): Find the first model with the solver. Add the negation of its assignment as another constraint and re-run the solver. Thus, the solver will find a different solution. Repeat until unsatisfiable.

In [1]:
def count_models_with_solver(solver, variables):
    solver.push() # as we will add further assertions to solver, checkpoint current state
    solutions = 0
    while solver.check() == sat:
        solutions = solutions + 1
        # Invert at least one variable to get a different solution:
        solver.add(Or([Not(x) if is_true(solver.model()[x]) else x for x in variables]))
    solver.pop() # restore solver to previous state
    return solutions

import itertools

def count_models_by_enumeration(solver, variables):
    solutions = 0
    for assignment in itertools.product([False, True], repeat = len(variables)): # all combinations
        satisfied = True
        for assertion in solver.assertions():
            if is_false(simplify(substitute(assertion, list(zip(variables, [BoolVal(x) for x in assignment]))))):
                satisfied = False
                break
        if satisfied: solutions = solutions + 1
    return solutions

We try both approaches with a small propositional formula with 10 variables, using an AND constraint as well as an OR constraint.

In [2]:
import time
from z3 import *

def benchmark(name, function, solver, variables):
    print('--'+ name + ' approach--')
    start_time = time.perf_counter()
    print('Number of models: ' + str(function(solver, variables)))
    end_time = time.perf_counter()
    print('Time: ' + str(round(end_time - start_time, 2)) + ' s')


x = Bools(' '.join('x' + str(i) for i in range(10)))
solver = Solver()

print('## OR formula ##')
solver.add(Or(x))
benchmark('Enumeration-based', count_models_by_enumeration, solver, x)
benchmark('Solver-based', count_models_with_solver, solver, x)

print('## AND formula ##')
solver.reset()
solver.add(And(x))
benchmark('Enumeration-based', count_models_by_enumeration, solver, x)
benchmark('Solver-based', count_models_with_solver, solver, x)

## OR formula ##
--Enumeration-based approach--
Number of models: 1023
Time: 0.46 s
--Solver-based approach--
Number of models: 1023
Time: 1.41 s
## AND formula ##
--Enumeration-based approach--
Number of models: 1
Time: 0.43 s
--Solver-based approach--
Number of models: 1
Time: 0.0 s


The enumeration-based approach has to evaluate the same number of variables for AND and OR formulas, i.e., all variables.
In our implementation, it would only benefit if there were multiple assertions, as it could abort earlier in some cases.
The solver-based approach is faster if there are few models and slower if there are many models.
Though it can build on previous solutions, finding a new model which also is different from the previous models actually becomes more difficult over time, the number of constraints grows.